In [1]:
import pandas as pd
import requests
import time
import os

/Users/ramazahosgor/Desktop/İTÜ/2024-2025-Bahar/yzv212e/hw1/hwProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
def fetch_nasa_power_data_for_point(lat, lon, start="2000-01-01", end="2024-12-31"):
    parameters = [
        "ALLSKY_SFC_SW_DWN",   # Surface Shortwave Downward Irradiance
        "ALLSKY_KT",           # Clearness Index
        "CLOUD_AMT",           # Cloud Amount
        "PRECTOTCORR"          # Precipitation
    ]

    url = (
        f"https://power.larc.nasa.gov/api/temporal/monthly/point"
        f"?start={start.replace('-', '')}&end={end.replace('-', '')}"
        f"&latitude={lat}&longitude={lon}"
        f"&community=AG"
        f"&parameters={','.join(parameters)}"
        f"&format=JSON"
    )

    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()
        records = data["properties"]["parameter"]
        dates = list(records[parameters[0]].keys())

        results = []
        for date in dates:
            entry = {"lat": lat, "lon": lon, "date": date}
            for param in parameters:
                entry[param] = records[param][date]
            results.append(entry)

        return pd.DataFrame(results)

    except Exception as e:
        print(f"Error fetching data for ({lat}, {lon}): {e}")
        return pd.DataFrame()

In [3]:
df = fetch_nasa_power_data_for_point(42.002, 28.254, start="2000", end="2024")
print(df.head())
df.to_csv("nasa_test_point.csv", index=False)

      lat     lon    date  ALLSKY_SFC_SW_DWN  ALLSKY_KT  CLOUD_AMT  \
0  42.002  28.254  200001               4.56     -999.0      80.83   
1  42.002  28.254  200002               9.38     -999.0      61.96   
2  42.002  28.254  200003              14.06     -999.0      61.32   
3  42.002  28.254  200004              18.50     -999.0      70.18   
4  42.002  28.254  200005              24.85     -999.0      33.43   

   PRECTOTCORR  
0         1.67  
1         2.31  
2         1.62  
3         1.81  
4         1.24  


In [6]:
df = pd.read_csv("assets/istanbul_filtered_grid.csv")

batch_size = 100
total_points = len(df)

output_dir = "assets/nasa_datas"
os.makedirs(output_dir, exist_ok=True)

for batch_start in range(0, total_points, batch_size):
    batch_end = min(batch_start + batch_size, total_points)
    batch = df.iloc[batch_start:batch_end]

    all_results = []
    print(f"\n🔄 Processing block {batch_start}–{batch_end - 1}")

    for i, row in batch.iterrows():
        lat, lon = row["lat"], row["lon"]
        print(f"→ Fetching point {i+1}/{total_points}: ({lat}, {lon})")
        
        data = fetch_nasa_power_data_for_point(lat, lon, start="2020", end="2023")
        if not data.empty:
            all_results.append(data)
        time.sleep(0.5)

    if all_results:
        block_df = pd.concat(all_results, ignore_index=True)
        block_index = (batch_start // batch_size) + 1
        output_file = os.path.join(output_dir, f"nasa_block_{block_index}.csv")
        block_df.to_csv(output_file, index=False)
        print(f"✅ Saved block to {output_file}")
    else:
        print(f"⚠️ No data collected for block {batch_start}–{batch_end - 1}")



🔄 Processing block 0–99
→ Fetching point 1/973: (40.82, 29.29)
→ Fetching point 2/973: (40.82, 29.32)
→ Fetching point 3/973: (40.84, 29.29)
→ Fetching point 4/973: (40.84, 29.32)
→ Fetching point 5/973: (40.84, 29.35)
→ Fetching point 6/973: (40.86, 29.26)
→ Fetching point 7/973: (40.86, 29.29)
→ Fetching point 8/973: (40.86, 29.32)
→ Fetching point 9/973: (40.86, 29.35)
→ Fetching point 10/973: (40.88, 29.2)
→ Fetching point 11/973: (40.88, 29.23)
→ Fetching point 12/973: (40.88, 29.26)
→ Fetching point 13/973: (40.88, 29.29)
→ Fetching point 14/973: (40.88, 29.32)
→ Fetching point 15/973: (40.88, 29.35)
→ Fetching point 16/973: (40.88, 29.38)
→ Fetching point 17/973: (40.88, 29.41)
→ Fetching point 18/973: (40.9, 29.14)
→ Fetching point 19/973: (40.9, 29.17)
→ Fetching point 20/973: (40.9, 29.2)
→ Fetching point 21/973: (40.9, 29.23)
→ Fetching point 22/973: (40.9, 29.26)
→ Fetching point 23/973: (40.9, 29.29)
→ Fetching point 24/973: (40.9, 29.32)
→ Fetching point 25/973: (40.9, 2